# Write the necessary weather data from the given .csv file

In [2]:
# imports -- change ip and fingerprint between hosts
import pandas as pd
import pyexasol

host = '192.168.56.102/08A2E50E38E949B851118C806E6A98B3EB37BB575015364670FBE8D0B4459EE1:8563'   # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

#### Read in CSV

In [3]:
df = pd.read_csv('./../Data/Stormdata_2006.csv', encoding='iso-8859-1')
print(df.shape)
print(df.columns)

# Convert start and end dates to date time
df['END_DATE_TIME'] = pd.to_datetime(df['END_DATE_TIME'])
df['BEGIN_DATE_TIME'] = pd.to_datetime(df['BEGIN_DATE_TIME'])

print(df.shape)
print("min:", min(df['BEGIN_DATE_TIME']), "\nmax:", max(df['END_DATE_TIME']))

# Remove those ending after our AOL database - Those before could be ok?
df = df[df['END_DATE_TIME'] <= '2006-06-01 00:00:00']

print(df.shape)
print("min:", min(df['BEGIN_DATE_TIME']), "\nmax:", max(df['END_DATE_TIME']))

(48595, 58)
Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'LAST_MOD_DATE',
       'LAST_MOD_TIME', 'LAST_CERT_DATE', 'LAST_CERT_TIME', 'LAST_MOD',
       'LAST_CERT', 'ADDCORR_FLG', 'ADDCORR_DATE'],
      dtype='obje

#### Drop irrelevant columns

In [4]:
# Keep: 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY', 'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS' 
keep = ['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY', 'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EPISODE_ID', 'EVENT_ID']

df = df.loc[:, keep]

print(df.columns)
print(df.head)

# Fix year/month and naming oddness
df['BEGIN_MONTH'] = df['BEGIN_YEARMONTH'] - 200600
df['END_MONTH'] = df['END_YEARMONTH'] - 200600
df['REGION'] = df['STATE'].str[0].str.upper() + df['STATE'].str[1:].str.lower() # This contains all the spatial information we need- writing the state name was buggy
df['EVENT_TYPE'] = df['EVENT_TYPE'].astype(str)

df = df.drop(columns= ['BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE'])

print(df.columns)
print(df.head)


Index(['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY',
       'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EPISODE_ID',
       'EVENT_ID'],
      dtype='object')
<bound method NDFrame.head of           BEGIN_DATE_TIME       END_DATE_TIME                EVENT_TYPE  \
0     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
1     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
2     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
3     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
4     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
...                   ...                 ...                       ...   
25729 2006-05-31 21:10:00 2006-05-31 22:15:00  Marine Thunderstorm Wind   
25730 2006-05-31 22:00:00 2006-05-31 22:00

#### Clean up the damage cols

In [5]:
def convert_abbreviated_string(value):
    # Check for 'k' (thousand), 'm' (million), 'b' (billion)
    if isinstance(value, str):
        print(value)
        if 'k' in value.lower():
            return float(value.replace('k', '').replace('K', '')) * 1000
        elif 'm' in value.lower():
            return float(value.replace('m', '').replace('M', '')) * 1000000
        elif 'b' in value.lower():
            return float(value.replace('b', '').replace('B', '')) * 1000000000
        else:
            # If no abbreviation, just return the float version of the number
            try:
                return float(value)
            except ValueError:
                return None  # or handle invalid strings as needed
    return value

In [6]:

# Print the unique columns and the one anomaly
print(df[df['DAMAGE_PROPERTY'].notna()]['DAMAGE_PROPERTY'].unique())
print(sum(df[df['DAMAGE_PROPERTY'].notna()]['DAMAGE_PROPERTY']=='K'))

print(df[df['DAMAGE_CROPS'].notna()]['DAMAGE_CROPS'].unique())

['900K' '4.9M' '2M' '3.2M' '4.5M' '700K' '100K' '1M' '15M' '165K' '243K'
 '524K' '623K' '1.2M' '5M' '115B' '104M' '0' '22M' '2.5M' '8.8M' '108M'
 '70K' '20K' '50K' '35K' '36K' '87K' '15K' '175K' '0K' '500K' '200K'
 '800K' '1K' '10K' '2K' '25K' '71K' '1.4M' '5.5M' '.1K' '75K' '5K' '45K'
 '55K' '.5K' '3K' '750K' '4M' '250K' '150K' '350K' '8K' '300K' '400K' '7K'
 '12K' '80K' '2.9M' '600K' '130K' '30K' '90K' '450K' '4K' '1.5K' '65K'
 '60K' '40K' '.25M' '120K' '.1M' '.5M' '160K' '16K' '2.5K' '.05K' '81K'
 '9K' '7.5M' '230K' '125K' '94.5K' '380K' '14K' '1.6M' '6K' '4.5K' '.01K'
 '1.5M' '85K' '11K' '168K' '357K' '.2K' '.25K' '10M' '.21K' '.17K' '8M'
 '19.9M' '30M' '3M' '850K' '50M' '650K' '3.5K' '126K' '265K' '.86K' '27K'
 '34K' '.75K' '190K' '240K' '43K' '210K' '21K' '.3K' '590K' '164K' '1.1M'
 '.04M' '.01M' '.15K' '.85K' '294K' '632K' '245K' '146K' '1.37M' '1.59M'
 '932K' '237K' '975K' '887K' '113K' '1.34M' '1.14M' '1.32M' '109K' '177K'
 '996K' '552K' '100M' '7M' '14.4M' '11.5M' '18K' '25M'

#### Modify the original DF

In [7]:
# Clean Up the damage property/crops data
df['DAMAGE_PROPERTY'] = df[df['DAMAGE_PROPERTY'] == 'K']['DAMAGE_PROPERTY'] = 1000  # Reporting error

# Get the string abbr. out
df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].replace("", float('nan')).apply(convert_abbreviated_string)
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].replace("", float('nan')).apply(convert_abbreviated_string)


1B
32.5M
3M
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
750K
0
0
0
0
0
0
0
0
10K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1.3M
0
0K
0
0
0
0
0
0
0
0
1M
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1M
0
0
0
0
0
0
0
0
0
10K
10K
0
0
30K
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
50K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0
3K
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15K
0
0
0
0
0
0
500K
300M
0
10K
2K
5K
0
0
0
30K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3.9M
1.52M
4.02M
2.5M
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0
0K
0K
0K
0K
0K
1K
112.5K
0
0
0
0
0
0
200M
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
500K
0
0
0
250K
0
0
0
0
5K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
100K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


#### Create The DB Schema on exasol

In [8]:
create_table_query = '''
CREATE TABLE AOL_SCHEMA.WEATHER_EVENTS (
    BEGIN_DATE_TIME TIMESTAMP,
    END_DATE_TIME TIMESTAMP,
    EVENT_TYPE VARCHAR(100),
    BEGIN_DAY INTEGER,
    END_DAY INTEGER,
    STATE_FIPS INTEGER,
    INJURIES_DIRECT INTEGER,
    INJURIES_INDIRECT INTEGER,
    DEATHS_DIRECT INTEGER,
    DEATHS_INDIRECT INTEGER,
    DAMAGE_PROPERTY INTEGER,
    DAMAGE_CROPS FLOAT,
    EPISODE_ID INT,
    EVENT_ID INT NOT NULL,
    BEGIN_MONTH INTEGER,
    END_MONTH INTEGER,
    REGION VARCHAR(100),
    PRIMARY KEY (EVENT_ID)
)
'''
conn.execute("DROP TABLE AOL_SCHEMA.WEATHER_EVENTS")
conn.execute(create_table_query)

<ExaStatement session_id=1816698514097591369 stmt_idx=2>

In [9]:
query = "DESCRIBE AOL_SCHEMA.WEATHER_EVENTS"  # Replace with your schema and table name
result = conn.execute(query)

# Fetch and print the results
for row in result:
    print(row)

('BEGIN_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('END_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('EVENT_TYPE', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('BEGIN_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('STATE_FIPS', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_PROPERTY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_CROPS', 'DOUBLE', 'TRUE', 'FALSE')
('EPISODE_ID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('EVENT_ID', 'DECIMAL(18,0)', 'FALSE', 'FALSE')
('BEGIN_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('REGION', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')


#### Write the data to the database

In [10]:
df.columns

Index(['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY',
       'END_DAY', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'EPISODE_ID', 'EVENT_ID', 'BEGIN_MONTH', 'END_MONTH', 'REGION'],
      dtype='object')

In [11]:
# Identify duplicates based on a specific column (e.g., 'EVENT_ID')
duplicates_df = df[df.duplicated(subset='EPISODE_ID', keep=False)]

# Display the duplicates
print(duplicates_df)


          BEGIN_DATE_TIME       END_DATE_TIME                EVENT_TYPE  \
0     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
1     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
2     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
3     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
4     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
...                   ...                 ...                       ...   
25728 2006-05-31 21:10:00 2006-05-31 22:10:00  Marine Thunderstorm Wind   
25729 2006-05-31 21:10:00 2006-05-31 22:15:00  Marine Thunderstorm Wind   
25730 2006-05-31 22:00:00 2006-05-31 22:00:00         Thunderstorm Wind   
25731 2006-05-31 22:05:00 2006-05-31 22:05:00         Thunderstorm Wind   
25732 2006-05-31 23:05:00 2006-05-31 23:05:00         Thunderstorm Wind   

       BEGIN_DAY  END_DAY  STATE_FIPS  INJURIES_DIRECT  INJURIES_INDIRECT  \
0              1      

In [12]:
df[df['EPISODE_ID']==202408]

BEGIN_DATE_TIME       END_DATE_TIME EVENT_TYPE  BEGIN_DAY  END_DAY  \
0        2006-01-01 2006-01-31 23:59:00    Drought          1       31   
201      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
202      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
203      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
204      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
291      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
292      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
293      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
295      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
312      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
313      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
314      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
315      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
316      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
317      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
318      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
319      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
320      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
321      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
322      2006-01-01 2006-01-31 23:59:00    Drought          1       31   
323      2006-01-01 2006-01-31 23:59:00    Drought          1       31   

     STATE_FIPS  INJURIES_DIRECT  INJURIES_INDIRECT  DEATHS_DIRECT  \
0             8                0                  0              0   
201           8                0                  0              0   
202           8                0                  0              0   
203           8                0                  0              0   
204           8                0                  0              0   
291           8                0                  0              0   
292           8                0                  0              0   
293           8                0                  0              0   
295           8                0                  0              0   
312           8                0                  0              0   
313           8                0                  0              0   
314           8                0                  0              0   
315           8                0                  0              0   
316           8                0                  0              0   
317           8                0                  0              0   
318           8                0                  0              0   
319           8                0                  0              0   
320           8                0                  0              0   
321           8                0                  0              0   
322           8                0                  0              0   
323           8                0                  0              0   

     DEATHS_INDIRECT  DAMAGE_PROPERTY  DAMAGE_CROPS  EPISODE_ID  EVENT_ID  \
0                  0             1000           NaN      202408   5482479   
201                0             1000           NaN      202408   5482474   
202                0             1000           NaN      202408   5482473   
203                0             1000           NaN      202408   5482472   
204                0             1000           NaN      202408   5482471   
291                0             1000           NaN      202408   5482470   
292                0             1000           NaN      202408   5482469   
293                0             1000           NaN      202408   5482467   
295                0             1000           NaN      202408   5482468   
312                0             1000           NaN      202408 

In [13]:

# Replace NaN values with None
df = df.where(pd.notnull(df), None)
print(df.shape)

# Remove duplicates based on all columns except 'EVENT_ID'
columns_except_event_id = df.columns[df.columns != 'EVENT_ID']

# Drop duplicates based on all columns except 'EVENT_ID'
distinct_df = df.drop_duplicates(subset=columns_except_event_id)

# Check the result
print(distinct_df.shape)


(25734, 17)
(19882, 17)


In [14]:
conn.import_from_pandas(distinct_df, table=('AOL_SCHEMA', 'WEATHER_EVENTS'))

In [15]:
conn.close()